In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Continuation into other Riemann sheets

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import Math

from ampform.dynamics.form_factor import FormFactor
from ampform.dynamics.phasespace import ChewMandelstamIntegral, PhaseSpaceFactor
from ampform.io import aslatex, improve_latex_rendering

improve_latex_rendering()
plt.rc("font", size=14)
warnings.filterwarnings("ignore")

## $T$ matrix definition

According to [PDG, Rev. Resonances](https://pdg.lbl.gov/2024/reviews/rpp2024-rev-resonances.pdf) the scattering amplitude is given by:

$$
M = n \left( I - i K \rho n^2 \right)^{-1} K n
$$

where $n$ is the barrier factor matrix and $\rho $ the phase space factor matrix. Both are diagonal matrices containing the respective factors for each channel. 
The factor $i\rho n^2$ is replaced by the once-subtracted Chew--Mandelstam dispersion integral $\Sigma^l$.

In [ ]:
class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):
        return printer._print(self.args[0])


n = 2
I = sp.Identity(n)
K = sp.MatrixSymbol("K", n, n)
CM = DiagonalMatrix(sp.MatrixSymbol(R"\Sigma^l", n, n))
FF = DiagonalMatrix(sp.MatrixSymbol(R"n", n, n))
Math(aslatex({CM: CM.as_explicit()}))

In [ ]:
T1 = FF * (I + K * CM).inv() * K * FF
T1

In [ ]:
T1.as_explicit()[0, 0].simplify(doit=False)

In [ ]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
g1 = sp.Symbol(R"g^{0}_1")
g2 = sp.Symbol(R"g^{0}_2")

In [ ]:
parameters = {
    ma1: 1.0,
    mb1: 1.5,
    ma2: 1.5,
    mb2: 2.0,
    m0: 3.0,
    g1: 1,
    g2: 1,
}
Math(aslatex(parameters))

## Symbolic dispersion integral 

In [ ]:
s, m1, m2, L = sp.symbols("s, m1, m2, L", nonnegative=True)
L_val = 1

In [ ]:
integral_expr = ChewMandelstamIntegral(s, m1, m2, L)
integral_expr.doit(deep=False)

## $T$ matrix parametrization

In [ ]:
T1_expressions = {
    K[0, 0]: (g1 * g2 * m0) / (s - m0**2),
    K[0, 1]: (g1 * g2 * m0) / (s - m0**2),
    K[1, 0]: (g1 * g2 * m0) / (s - m0**2),
    K[1, 1]: (g2 * g2 * m0) / (s - m0**2),
    FF[0, 0]: FormFactor(s, ma1, mb1, angular_momentum=L_val),
    FF[1, 1]: FormFactor(s, ma2, mb2, angular_momentum=L_val),
}
Math(aslatex(T1_expressions))

In [ ]:
T1_expr = T1.as_explicit().xreplace(T1_expressions)
T1_expr[0, 0].simplify(doit=False)

## Sheets II, III, and IV

In the case of two channels, there are four Riemann sheets. The first sheet is physical and three unphysical ones. The physical sheet is calculated using the analytic solution of the Chew-Mandelstam function.
The other sheets are reached by adding the discontinuity across the branch cut:
For higher $l$:
$$
\operatorname{Disc} \varSigma_\ell(s) \;=\; 2i\,\rho(s)\,n_\ell^2(s)
$$
E.g.:

$$
T^{II}=n\left(\mathbb{I} + K (\varSigma_\ell + 2 i \rho)\right)^{-1} K n
$$

Depending on the centre-of-mass energy, different Riemann sheets connect smoothly to the physical one. Therefore, two cases are studied: one where the resonance mass is above the threshold of the second and first channel, and another where the resonance mass is between the threshold of the first and second channel. For the 2 channel one gets:

$$
\begin{eqnarray}
\operatorname{Disc}_{\mathrm{I,II}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}\rho_1 n_{\ell,1}^2(s) & 0  \\ 0 & 0 \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,III}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}\rho_1 n_{\ell,1}^2(s) & 0  \\ 0 & \rho_2 n_{\ell,2}^2(s) \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,IV}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}0 & 0  \\ 0& \rho_2  n_{\ell,2}^2(s) \end{array}\right].
\end{eqnarray}
$$

In [ ]:
rho = DiagonalMatrix(sp.MatrixSymbol("rho", n, n))
Math(aslatex({rho: rho.as_explicit()}))

In [ ]:
T_unphysical = FF * (I + K * (CM + sp.I * 2 * rho * FF**2)).inv() * K * FF

In [ ]:
T_unphysical.as_explicit()[0, 0].simplify(doit=False)

In [ ]:
T2_expressions = {
    **T1_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: 0,
}
T3_expressions = {
    **T1_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}
T4_expressions = {
    **T1_expressions,
    rho[0, 0]: 0,
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}
T2_expr = T_unphysical.as_explicit().xreplace(T2_expressions)
T3_expr = T_unphysical.as_explicit().xreplace(T3_expressions)
T4_expr = T_unphysical.as_explicit().xreplace(T4_expressions)

## Plot

In [ ]:
symbols = (s, CM[0, 0], CM[1, 1])
T1_func = sp.lambdify(symbols, T1_expr[0, 0].subs(parameters).doit())
T2_func = sp.lambdify(symbols, T2_expr[0, 0].subs(parameters).doit())
T3_func = sp.lambdify(symbols, T3_expr[0, 0].subs(parameters).doit())
T4_func = sp.lambdify(symbols, T4_expr[0, 0].subs(parameters).doit())

integral_p_wave_func = sp.lambdify(
    [s, m1, m2, integral_expr.epsilon],
    integral_expr.subs(L, L_val).doit(),
)

In [ ]:
def evaluate(func, s_array):
    args = transform(s_array)
    return func(*args)


def transform(s):
    cm1_array = integral_p_wave_func(s, parameters[ma1], parameters[mb1], ε_integral)
    cm2_array = integral_p_wave_func(s, parameters[ma2], parameters[mb2], ε_integral)
    return s, cm1_array, cm2_array


ε_integral = 1e-8

In [ ]:
ε_cut = 0.02
x = np.linspace(0, 8, num=300)
y = np.linspace(ε_cut, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

In [ ]:
T1n = evaluate(T1_func, Zn**2)
T1p = evaluate(T1_func, Zp**2)

T2n = evaluate(T2_func, Zn**2)
T2p = evaluate(T2_func, Zp**2)

T3n = evaluate(T3_func, Zn**2)
T3p = evaluate(T3_func, Zp**2)

T4n = evaluate(T4_func, Zn**2)
T4p = evaluate(T4_func, Zp**2)

In [ ]:
fig, axes = plt.subplots(figsize=(11, 4), ncols=2, sharey=True)
ax1, ax2 = axes.ravel()
fig.subplots_adjust(wspace=0.1)
fig.suptitle(R"$s_{thr1}<s_{thr2}<m_{res}$", fontsize=20)
ax1.set_title("I and II", y=0.88)
ax2.set_title("I and III", y=0.88)
for ax in axes.ravel():
    ax.set_xlabel(R"$\mathrm{Re}\,\sqrt{s}$")
ax1.set_ylabel(R"$\mathrm{Im}\,\sqrt{s}$")

T_max = 2
style = dict(vmin=-T_max, vmax=+T_max, cmap="RdBu")
mesh = ax1.pcolormesh(X, Y, T1p.imag, **style)
ax1.pcolormesh(X, -Y, T2n.imag, **style)
ax2.pcolormesh(X, +Y, T1p.imag, **style)
ax2.pcolormesh(X, -Y, T3n.imag, **style)

s_thr1 = parameters[ma1] + parameters[mb1]
s_thr2 = parameters[ma2] + parameters[mb2]
linestyle = dict(ls="dotted", lw=1)
for ax in [ax1, ax2]:
    ax.axhline(0, c="black", **linestyle)
    ax.axvline(s_thr1, c="C0", **linestyle, label=R"$\sqrt{s_\mathrm{thr1}}$")
    ax.axvline(s_thr2, c="C1", **linestyle, label=R"$\sqrt{s_\mathrm{thr2}}$")

linestyle_res = dict(c="r", ls="dotted", label=R"$m_\mathrm{res}$")
for ax in [ax1, ax2]:
    ax.axvline(parameters[m0], **linestyle_res)

ax1.legend(fontsize=11, loc="lower left")
cbar = fig.colorbar(mesh, ax=axes)
cbar.ax.set_ylabel(R"$\mathrm{Im} T(s)$")
fig.show()

In [ ]:
fig, axes = plt.subplots(figsize=(12, 6), ncols=4, sharey=True)
ax1, ax2, ax3, ax4 = axes.ravel()

ax1.plot(x, T1n[0].imag, label=R"$T_\mathrm{I}(s-0i)$")
ax1.plot(x, T1p[0].imag, label=R"$T_\mathrm{I}(s+0i)$")
ax1.set_title(R"$T_\mathrm{I}$")

ax2.plot(x, T2n[0].imag, label=R"$T_\mathrm{II}(s-0i)$")
ax2.plot(x, T2p[0].imag, label=R"$T_\mathrm{II}(s+0i)$")
ax2.set_title(R"$T_\mathrm{II}$")

ax3.plot(x, T3n[0].imag, label=R"$T_\mathrm{III}(s-0i)$")
ax3.plot(x, T3p[0].imag, label=R"$T_\mathrm{III}(s+0i)$")
ax3.set_title(R"$T_\mathrm{III}$")

ax4.plot(x, T4n[0].imag, label=R"$T_\mathrm{IV}(s-0i)$")
ax4.plot(x, T4p[0].imag, label=R"$T_\mathrm{IV}(s+0i)$")
ax4.set_title(R"$T_\mathrm{IV}$")

for ax in axes:
    ax.legend()
    ax.set_xlabel(R"$\mathrm{Re}\,\sqrt{s}$")
    ax.set_ylim(-1, +1)
ax1.set_ylabel(R"$\mathrm{Im}\,T(s)$ (a.u.)")

fig.tight_layout()
plt.show()

In [ ]:
x_index = 100

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(Zp.imag[:, x_index], T1p[:, x_index].imag, label="T I +", c="red")
ax.plot(Zn.imag[:, x_index], T1n[:, x_index].imag, label="T I -", c="darkred")
ax.plot(Zp.imag[:, x_index], T2p[:, x_index].imag, label="T II +", c="blue")
ax.plot(Zn.imag[:, x_index], T2n[:, x_index].imag, label="T II -", c="darkblue")
ax.plot(Zp.imag[:, x_index], T3p[:, x_index].imag, label="T III +", c="lightgreen")
ax.plot(Zn.imag[:, x_index], T3n[:, x_index].imag, label="T III -", c="darkgreen")
ax.plot(Zp.imag[:, x_index], T4p[:, x_index].imag, label="T IV +", c="yellow")
ax.plot(Zn.imag[:, x_index], T4n[:, x_index].imag, label="T IV -", c="gold")
ax.set_xlabel(R"$\mathrm{Im}\,\sqrt{s}$")
ax.set_ylabel("Im(T)")
ax.set_xlim(-0.5, 0.5)
ax.legend()
plt.show()